<a href="https://colab.research.google.com/github/sscoconut64/Joey_DTSC3020_Fall2025/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [2]:
#1) #Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [3]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [4]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [17]:
# Q1 — Write your answer here
from io import StringIO

def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML."""
    # Wrap the HTML string in StringIO to avoid the FutureWarning
    dfs = pd.read_html(StringIO(html))
    for df in dfs:
        if df.shape[1] >= 3:
            return flatten_headers(df)
    return pd.DataFrame() # Return empty if no suitable table found

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids."""
    df.columns = [col.strip() for col in df.columns]

    # Define the expected column names and potential variations
    col_mapping = {
        'Country': ['Country'],
        'Alpha-2': ['Alpha-2', 'Alpha-2 code'],
        'Alpha-3': ['Alpha-3', 'Alpha-3 code'],
        'Numeric': ['Numeric']
    }

    cleaned_df = pd.DataFrame()
    for standard_col, possible_names in col_mapping.items():
        for name in possible_names:
            if name in df.columns:
                cleaned_df[standard_col] = df[name]
                break
        # If no possible name is found, create the column with NaNs
        if standard_col not in cleaned_df.columns:
             cleaned_df[standard_col] = pd.NA


    # Clean text columns
    for col in ['Country', 'Alpha-2', 'Alpha-3']:
        if col in cleaned_df.columns:
            cleaned_df[col] = cleaned_df[col].astype(str).str.strip()
            if col in ['Alpha-2', 'Alpha-3']:
                cleaned_df[col] = cleaned_df[col].str.upper()

    # Clean Numeric column
    if 'Numeric' in cleaned_df.columns:
        # Attempt to convert to numeric, coercing errors to NaN
        cleaned_df['Numeric'] = pd.to_numeric(cleaned_df['Numeric'], errors='coerce')
        # Convert to nullable integer type
        cleaned_df['Numeric'] = cleaned_df['Numeric'].astype('Int64')


    # Drop rows where essential columns might be missing after cleaning (e.g., Numeric)
    cleaned_df.dropna(subset=['Country', 'Alpha-2', 'Alpha-3', 'Numeric'], inplace=True)

    return cleaned_df


def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N."""
    if 'Numeric' in df.columns:
        return df.sort_values(by='Numeric', ascending=False).head(top)
    return df.head(top) # Return top rows without sorting if Numeric is missing

# --- Q1 Execution ---
q1_url = "https://www.iban.com/country-codes"
q1_html = fetch_html(q1_url)
q1_df = q1_read_table(q1_html)
q1_cleaned_df = q1_clean(q1_df)

# Save results to CSV
q1_cleaned_df.to_csv('data_q1.csv', index=False)
print("data_q1.csv saved.")

# Display Top-15
q1_top_15 = q1_sort_top(q1_cleaned_df)
print("Top-15 results for Question 1:")
display(q1_top_15)

data_q1.csv saved.
Top-15 results for Question 1:


,Country,Alpha-2,Alpha-3,Numeric
247,Zambia,ZM,ZMB,894
246,Yemen,YE,YEM,887
192,Samoa,WS,WSM,882
244,Wallis and Futuna,WF,WLF,876
240,Venezuela (Bolivarian Republic of),VE,VEN,862
238,Uzbekistan,UZ,UZB,860
237,Uruguay,UY,URY,858
35,Burkina Faso,BF,BFA,854
243,Virgin Islands (U.S.),VI,VIR,850
236,United States of America (the),US,USA,840


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [10]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [15]:
# Q2 — Write your answer here
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    """
    soup = BeautifulSoup(html, 'lxml')
    items = []
    for row in soup.select('tr.athing'):
        rank = row.select_one('.rank').get_text(strip=True).replace('.', '')
        title_link = row.select_one('.titleline a')
        title = title_link.get_text(strip=True) if title_link else ''
        link = title_link['href'] if title_link else ''

        subtext_row = row.find_next_sibling('tr')
        points = subtext_row.select_one('.score')
        points = points.get_text(strip=True).split()[0] if points else '0'

        comments = subtext_row.select_one('a[href^="item?id="]')
        comments = comments.get_text(strip=True).split()[0] if comments and 'comment' in comments.get_text() else '0'

        user = subtext_row.select_one('.hnuser')
        user = user.get_text(strip=True) if user else ''

        items.append({
            'rank': rank,
            'title': title,
            'link': link,
            'points': points,
            'comments': comments,
            'user': user
        })
    return pd.DataFrame(items)


def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values."""
    for col in ['rank', 'points', 'comments']:
        # Convert to numeric, coercing errors to 0
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    # Fill missing text fields
    for col in ['title', 'link', 'user']:
        df[col] = df[col].fillna('')

    return df


def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N."""
    return df.sort_values(by='points', ascending=False).head(top)

# --- Q2 Execution ---
q2_url = "https://news.ycombinator.com/"
q2_html = fetch_html(q2_url)
q2_df = q2_parse_items(q2_html)
q2_cleaned_df = q2_clean(q2_df)

# Save results to CSV
q2_cleaned_df.to_csv('data_q2.csv', index=False)
print("data_q2.csv saved.")

# Display Top-15
q2_top_15 = q2_sort_top(q2_cleaned_df)
print("Top-15 results sorted by points:")
display(q2_top_15)

data_q2.csv saved.
Top-15 results sorted by points:


,rank,title,link,points,comments,user
9,10,"Tiny electric motor can produce more than 1,00...",https://supercarblondie.com/electric-motor-yas...,524,0,chris_overseas
22,23,Why Nextcloud feels slow to use,https://ounapuu.ee/posts/2025/11/03/nextcloud-...,351,0,rpgbr
2,3,Ask HN: Who is hiring? (November 2025),item?id=45800465,269,0,whoishiring
13,14,The Case Against PGVector,https://alex-jacobs.com/posts/the-case-against...,254,0,tacoooooooo
18,19,WebAssembly (WASM) arch support for the Linux ...,https://github.com/joelseverin/linux-wasm,211,0,marcodiego
17,18,</> Htmx – The Fetch()ening,https://htmx.org/essays/the-fetchening/,209,0,leephillips
5,6,Learning to read Arthur Whitney's C to become ...,https://needleful.net/blog/2024/01/arthur_whit...,208,0,gudzpoz
15,16,A visualization of the RGB space covered by na...,https://codepen.io/meodai/full/zdgXJj/,200,0,BlankCanvas
20,21,First recording of a dying human brain shows w...,https://louisville.edu/medicine/news/first-eve...,182,0,thunderbong
26,27,Why we migrated from Python to Node.js,https://blog.yakkomajuri.com/blog/python-to-node,173,0,yakkomajuri
